<a href="https://colab.research.google.com/github/MuroriM/GEM-table-to-text-through-AMR/blob/main/gem_preprocessing_triples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
from datasets import load_dataset

import re
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import json

from google.colab import files

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-0fpzujey
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-0fpzujey
  Created wheel for datasets: filename=datasets-1.6.2.dev0-cp37-none-any.whl size=224908 sha256=b485a484f0c8d1078aecddac01737c309f0f7d45e849fca4d9bb45a74cd386ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-nol4yl0t/wheels/3e/af/ff/d1cdb5d0f9cff6eba2042a16b477ada497e23f1a3b6950b928
Successfully built datasets
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# find related verb form
def normalize_relation_verb_webnlg(relation):

    # modrelation = re.findall('[a-zA-Z][^A-Z]*', relation)[0]
    modrelation = relation.replace('_', ' ')
    # separate relations into multiple words where a capital letter mid-word is found
    modrelation = re.findall(r'[a-zA-Z][^A-Z]*', modrelation)
    
    #modrelation = re.sub(r'[^\w\s]','', relation)
    
    relation = modrelation

    # currently not normalizing words in multi-word relations: leads to a decrease in quality

    # if isinstance(relation, list): 
    #     final_relation = []
    #     for word in relation:
            
    #         lemmas = wn.lemmas(word)

    #         if lemmas:
    #             related_forms = [lemmas[i].derivationally_related_forms() for i in range(len(lemmas)) if lemmas[i].derivationally_related_forms()]

    #             if related_forms:
    #                 verb = related_forms[0][0].name()
    #                 related_verb = verb.lower()

               
    #             verb_forms = wn.synsets(word, pos="v")
                
    #             if verb_forms:
    #                 verb = verb_forms[0].lemmas()[0].name()
    #                 final_relation.append(verb.lower())
    #             elif related_forms: 
    #                 final_relation.append(related_verb)
    #         else:
    #             final_relation.append(word.lower())
    #     return 0, ' '.join(final_relation)



    if len(relation) == 1:
        relation = relation[0]
        lemmas = wn.lemmas(relation)

        if lemmas:
            related_forms = [lemmas[i].derivationally_related_forms() for i in range(len(lemmas)) if lemmas[i].derivationally_related_forms()]

            if related_forms:
                verb = related_forms[0][0].name()
                return 1, verb.lower()

            
            verb_forms = wn.synsets(relation, pos="v")
            if verb_forms:
                verb = verb_forms[0].lemmas()[0].name()
                return 1, verb.lower()

        return 0, relation.lower()   
     
    return 0, ' '.join(relation).lower()

In [ ]:
def normalize_triple_entity(entity):
    # modentity = re.sub(r'[^\w\s]','', entity)
    modentity = entity.replace('_', ' ')
    modentity = modentity.replace('"', '')
    
    return modentity

In [ ]:
def get_triples_text_pairs_webnlg(dframe, outfilename):
    processed_webnlg = []

    for i, row in dframe.iterrows():
        current_triples = []
        for j, triple in enumerate(row['tripleset']):
            found, verb = normalize_relation_verb_webnlg(triple[1])
            triple[0] = normalize_triple_entity(triple[0])
            triple[1] = verb
            triple[2] = normalize_triple_entity(triple[2])
            
            current_triples.append(" ".join(triple))
        processed_webnlg.append({"triple-sents": current_triples, "text": row['text']})

    with open(outfilename, 'w') as f:
      json.dump(processed_webnlg, f, ensure_ascii=False)

    files.download(outfilename)

In [ ]:
webnlg_data = load_dataset("gem", "web_nlg_en")

print(webnlg_data['train'][100])

Reusing dataset gem (/root/.cache/huggingface/datasets/gem/web_nlg_en/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4)


{'category': 'Airport', 'gem_id': 'web_nlg_en-train-100', 'gem_parent_id': 'web_nlg_en-train-100', 'input': ['Afonso_Pena_International_Airport | elevationAboveTheSeaLevel | 911.0'], 'references': [], 'target': 'Afonso Pena International airport is located 911 metres above sea level.', 'webnlg_id': 'train/Airport/1/Id42'}


In [ ]:
def get_triples_text_pairs_webnlg(data, outfilename):
    processed_webnlg = []

    for entry in data:
        current_triples = []
        
        for triple in entry['input']:

            triple = triple.split(" | ")
            
            found, verb = normalize_relation_verb_webnlg(triple[1])
            triple[0] = normalize_triple_entity(triple[0])
            triple[1] = verb
            triple[2] = normalize_triple_entity(triple[2])
            
            current_triples.append(" ".join(triple))
            
        processed_webnlg.append({"triple-sents": current_triples, "text": entry['target'], "gem_id": entry['gem_id'], "gem_parent_id": entry['gem_parent_id']})
        
    with open(outfilename, 'w') as f:
      json.dump(processed_webnlg, f, ensure_ascii=False)

    files.download(outfilename)

In [ ]:
get_triples_text_pairs_webnlg(webnlg_data['train'], 'gem_processed_webnlg_train.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
get_triples_text_pairs_webnlg(webnlg_data['validation'], 'gem_processed_webnlg_dev.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
get_triples_text_pairs_webnlg(webnlg_data['test'], 'gem_processed_webnlg_test.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>